In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, PReLU, Input, Activation

Using TensorFlow backend.


In [2]:
data = pd.read_csv('../input/titanic/train.csv')
data_t = pd.read_csv('../input/titanic/test.csv')

In [3]:
val = data['Age'].median()

def clean_data(data):
    data['Age'].fillna(val, inplace=True)
    data.drop('Cabin', inplace=True, axis=1)
    data['Embarked'].fillna('S', inplace=True)
    data['Sex'] = data['Sex'].map({'male': 0, 'female': 1})
    data['Title'] = data['Name'].str.split(', ', expand=True)[1].str.split('.', expand=True)[0]
    data['Is_Married'] = 0
    data['Is_Married'].loc[data['Title'] == 'Mrs'] = 1    
    return data

data = clean_data(data)

/opt/conda/lib/python3.6/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [4]:
mean_enc_cols = ['Embarked', 'Pclass', 'Title']
cols = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked', 'Title']
cat_cols = ['Is_Married']
encs, scales = [], []

for col in mean_enc_cols:
    enc = dict(data.groupby(col).agg({'Survived': 'mean'}).reset_index().values)
    encs.append(enc)
    data[col] = data[col].map(enc)
    
for col in cols:
    sc = StandardScaler()
    data[col] = sc.fit_transform(data[col].values.reshape(-1, 1))
    scales.append(sc)

In [5]:
X = data[cols+cat_cols].values
y = data['Survived'].values
print(X.shape, y.shape)

(891, 9) (891,)


In [6]:
model = Sequential()
model.add(Dense(48, input_shape=(X.shape[1],)))
model.add(PReLU())
model.add(Dense(32))
model.add(PReLU())
model.add(Dense(1,activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [7]:
model.fit(X, y, batch_size=64, epochs=32)

Epoch 1/32
891/891 [==============================] - 1s 1ms/step - loss: 0.6740 - accuracy: 0.5780
Epoch 2/32
891/891 [==============================] - 0s 56us/step - loss: 0.5596 - accuracy: 0.7946
Epoch 3/32
891/891 [==============================] - 0s 54us/step - loss: 0.4842 - accuracy: 0.8103
Epoch 4/32
891/891 [==============================] - 0s 62us/step - loss: 0.4410 - accuracy: 0.8204
Epoch 5/32
891/891 [==============================] - 0s 75us/step - loss: 0.4246 - accuracy: 0.8227
Epoch 6/32
891/891 [==============================] - 0s 65us/step - loss: 0.4158 - accuracy: 0.8227
Epoch 7/32
891/891 [==============================] - 0s 76us/step - loss: 0.4089 - accuracy: 0.8283
Epoch 8/32
891/891 [==============================] - 0s 54us/step - loss: 0.4054 - accuracy: 0.8283
Epoch 9/32
891/891 [==============================] - 0s 54us/step - loss: 0.4016 - accuracy: 0.8283
Epoch 10/32
891/891 [==============================] - 0s 52us/step - loss: 0.3986 - accurac

In [8]:
data_t = clean_data(data_t)

for i, col in enumerate(mean_enc_cols):
    data_t[col] = data_t[col].map(encs[i])
    
for i, col in enumerate(cols):
    data_t[col] = scales[i].transform(data_t[col].values.reshape(-1, 1))

X_test = data_t[cols+cat_cols].values

/opt/conda/lib/python3.6/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [9]:
ops = model.predict(X_test).ravel()
fops = np.zeros(X_test.shape[0], dtype=np.int32)
fops[ops > 0.5] = 1

sub = pd.DataFrame({'PassengerId': data_t['PassengerId'], 'Survived': fops})
sub.to_csv('sub.csv', index=False)